In [0]:
# 03_tile_stats_v2.py
from pyspark.sql import functions as F
from trimble_geospatial_demo_utils.site_lock import acquire_site_lock, release_site_lock

# ==================================================
# Unity Catalog context
# ==================================================
spark.sql("USE CATALOG main")
spark.sql("USE SCHEMA demo")

# ==================================================
# Read Job Parameters
# ==================================================
dbutils.widgets.text("siteId", "", "Site ID")

SITE_ID = dbutils.widgets.get("siteId")

# Validate required parameters
if not SITE_ID:
    raise ValueError("Missing required job parameter: siteId")

print(f"🏗️  Site ID: {SITE_ID}")

# ==================================================
# CONFIG
# ==================================================
INPUT_TABLE  = "processed_points_tiled_v2"
OUTPUT_TABLE = "tile_stats_v2"
OUTPUT_PATH  = "abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/tile_stats_v2"

# Water classification code (common LAS: 9). Adjust if your dataset differs.
WATER_CLASS = 9

# Threshold to consider a tile "mostly water"
MOSTLY_WATER_THRESHOLD = 0.60

# ==================================================
# Job identity (for locking / audit)
# ==================================================
JOB_RUN_ID = spark.conf.get("spark.databricks.job.runId", "manual-notebook")

# ==================================================
# Acquire site-level lock (latest snapshot semantics)
# ==================================================
acquire_site_lock(
    spark=spark,
    site_id=SITE_ID,
    locked_by=JOB_RUN_ID,
    ttl_minutes=60
)

try:
    # ==================================================
    # 1) Read latest processed snapshot for this site
    # ==================================================
    df_points = (
        spark.table(INPUT_TABLE)
             .filter(F.col("siteId") == SITE_ID)
    )

    if df_points.rdd.isEmpty():
        raise RuntimeError(f"No data found for siteId={SITE_ID} in {INPUT_TABLE}")

    # ==================================================
    # 2) Aggregate per tile (one row per tile)
    # ==================================================
    df_tile_stats = (
        df_points
        .groupBy("siteId", "tileId")
        .agg(
            # Core counts
            F.count("*").alias("pointCount"),

            # Z stats
            F.min("z").alias("minZ"),
            F.max("z").alias("maxZ"),
            F.expr("percentile_approx(z, 0.50)").alias("z_p50"),
            F.expr("percentile_approx(z, 0.95)").alias("z_p95"),
            F.expr("percentile_approx(z, 0.99)").alias("z_p99"),

            # Intensity stats
            F.expr("percentile_approx(intensity, 0.50)").alias("intensity_p50"),
            F.expr("percentile_approx(intensity, 0.95)").alias("intensity_p95"),

            # Water tracking (count of points classified as water)
            F.sum(F.when(F.col("classification") == F.lit(WATER_CLASS), F.lit(1)).otherwise(F.lit(0))).alias("waterPointCount"),
        )
        # Water ratio derived from counts (avoid divide-by-zero)
        .withColumn(
            "waterPointRatio",
            F.when(F.col("pointCount") > 0, F.col("waterPointCount") / F.col("pointCount")).otherwise(F.lit(0.0))
        )
        # Mostly-water flag for routing
        .withColumn(
            "isMostlyWater",
            (F.col("waterPointRatio") >= F.lit(MOSTLY_WATER_THRESHOLD)).cast("boolean")
        )
        .withColumn("computedAt", F.current_timestamp())
    )

    # ==================================================
    # 3) Safety check: ensure only one siteId
    # ==================================================
    if df_tile_stats.select("siteId").distinct().count() != 1:
        raise RuntimeError("Aggregation output contains multiple siteId values")

    # ==================================================
    # 4) Write latest snapshot (replace entire site)
    # ==================================================
    (
        df_tile_stats.write
            .format("delta")
            .mode("overwrite")
            .option("replaceWhere", f"siteId = '{SITE_ID}'")
            .option("path", OUTPUT_PATH)
            .partitionBy("siteId", "tileId")
            .saveAsTable(OUTPUT_TABLE)
    )

    # ==================================================
    # 5) Verification
    # ==================================================
    print("\n=== Verify tile_stats_v2 ===")
    spark.sql(f"""
        SELECT
          siteId,
          COUNT(*)                          AS tiles,
          SUM(pointCount)                   AS totalPoints,
          SUM(waterPointCount)              AS totalWaterPoints,
          AVG(waterPointRatio)              AS avgWaterRatio,
          SUM(CASE WHEN isMostlyWater THEN 1 ELSE 0 END) AS mostlyWaterTiles
        FROM {OUTPUT_TABLE}
        WHERE siteId = '{SITE_ID}'
        GROUP BY siteId
    """).show(truncate=False)

    print("✅ tile_stats_v2 written successfully (with water tracking)")

finally:
    # ==================================================
    # Release site-level lock
    # ==================================================
    release_site_lock(
        spark=spark,
        site_id=SITE_ID,
        locked_by=JOB_RUN_ID
    )